# Model baseline exploration

This notebook explores different models, prompts and inputs to find a good baseline. 

We are looking for a baseline that follows conventions laid out in /Users/esmahoney/Projects/klartext/klartext/notebooks/00_klartext_overview.ipynb 

The strategy is as follows: 
Step 1. Setup & Configuration - initialize Groq and list available models that are multilingual. 
2. Load dataset from 

| Dataset | Purpose | Metrics |
|---------|---------|---------|
| ASSET (EN) | Multi-reference evaluation | SARI, BLEU, FKGL |
| TurkCorpus (EN) | Multi-reference evaluation | SARI, BLEU |
| DEplain-test (DE) | German evaluation | SARI, BLEU |

In this step we will evaluate models and provide a simplicity score from 1-10 

Step 2. Exploration A: Prompt Strategy
Step 3. Evaluation



# Step 1: Setup & Configuration

In [1]:
# Environment setup
import os
import time
import json
from pathlib import Path 
from dotenv import load_dotenv
from groq import Groq
from datasets import load_dataset
import pandas as pd

#Load API keys from project root .env file
# The notebook is in /notebooks/, so we go up one level to find .env
project_root = Path(__file__).parent.parent if '__file__' in dir() else Path.cwd().parent
env_path = project_root / ".env"

# Alternative: use explicit path
env_path = Path("/Users/esmahoney/Projects/klartext/klartext/.env")

print(f"Looking for .env at: {env_path}")
print(f"File exists: {env_path.exists()}")

# Load from explicit path
load_dotenv(dotenv_path=env_path)

api_key = os.getenv("GROQ_API_KEY")
if api_key:
    print(f"✅ API key loaded (starts with: {api_key[:8]}...)")
    client = Groq(api_key=api_key)
else:
    print("❌ GROQ_API_KEY still not found!")
    print(f"   Please ensure {env_path} contains: GROQ_API_KEY=gsk_...")

/Users/esmahoney/Projects/klartext/klartext/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Looking for .env at: /Users/esmahoney/Projects/klartext/klartext/.env
File exists: True
✅ API key loaded (starts with: gsk_VX1z...)


In [5]:
# List Groq models and filter for multilingual capability
models = client.models.list()

print("🔍 Available Models on Groq:\n")
print(f"{'Model ID':<50} {'Context':<10} {'Owner':<15}")
print("-" * 75)

for m in models.data:
    print(f"{m.id:<50} {m.context_window:<10} {m.owned_by:<15}")

🔍 Available Models on Groq:

Model ID                                           Context    Owner          
---------------------------------------------------------------------------
whisper-large-v3                                   448        OpenAI         
meta-llama/llama-prompt-guard-2-22m                512        Meta           
meta-llama/llama-4-maverick-17b-128e-instruct      131072     Meta           
playai-tts-arabic                                  8192       PlayAI         
whisper-large-v3-turbo                             448        OpenAI         
meta-llama/llama-prompt-guard-2-86m                512        Meta           
allam-2-7b                                         4096       SDAIA          
llama-3.3-70b-versatile                            131072     Meta           
meta-llama/llama-4-scout-17b-16e-instruct          131072     Meta           
meta-llama/llama-guard-4-12b                       131072     Meta           
openai/gpt-oss-safeguard-20b         

Looking at these models and doing some research, we know that the best ones are:

* qwen/qwen3-32b - specifically designed for multilingual tasks and has excellent German support. Strong instruction-following. Good balance of quality and speed. Best for DE/EN simplification
* meta-llama/llama-4-maverick-17b-128e-instruct - Newest Llama 4 architecture. Mixture-of-Experts (MoE) means it's fast despite being capable. Good to compare against the Llama 3 models being tested for improvements.
* moonshotai/kimi-k2-instruct - Different architecture/training from Meta/Alibaba. Known for strong reasoning and following complex instructions. Gives diversity to the baseline comparison. Good for testing a non-mainstream model for comparison.


For follow-up additional exploration later: 
* groq/compound - This is Groq's own compound model — it likely routes requests to the best available model automatically. Interesting to see if it optimizes for your task. Best for "Let the system choose" baseline.     
* allam-2-7b - Much smaller/faster model. Test if a lightweight model can handle simplification adequately — important for cost/speed trade-offs. Note: Primarily trained for Arabic; may have limited DE/EN quality.

In [8]:
MODELS_TO_TEST = [
    "qwen/qwen3-32b",                              # Strong multilingual, good DE/EN
    "meta-llama/llama-4-maverick-17b-128e-instruct",  # Latest Llama 4, 128K context
    "moonshotai/kimi-k2-instruct",                 # Different architecture, strong reasoning
]

# Verify models are available
print("🔍 Verifying models are available on Groq:")
available_model_ids = {m.id for m in client.models.list().data}

for model in MODELS_TO_TEST:
    status = "✅" if model in available_model_ids else "❌ NOT FOUND"
    print(f"  {status} {model}")

LEVELS = ["very_easy", "easy", "medium"]

🔍 Verifying models are available on Groq:
  ✅ qwen/qwen3-32b
  ✅ meta-llama/llama-4-maverick-17b-128e-instruct
  ✅ moonshotai/kimi-k2-instruct


Step 2: Load Evaluation Datasets
For this we will be using datasets from huggingface. Some of them only have full split available - meaning it is not available in test/train/validation splits and is only released in 1 big chunk, but this isn't a problem for our eval since we are not training.

ASSET huggingface.co/datasets/asset, contains 2,359 sentences × 10 simplified references each
TurkCorpus huggingface.co/datasets/turkcorpus, contains 2,359 sentences × 8 simplified references each
DEplain-test huggingface.co/datasets/DEplain/DEplain-web, ~600K German sentence pairs

In [ ]:
from datasets import load_dataset

# ============================================================
# ASSET - English simplification benchmark
# ============================================================
# The "ratings" config only has a "full" split
# Use "simplification" config for train/validation/test splits

print("Loading ASSET...")
try:
    # Option 1: Use "simplification" config (has validation split)
    asset_data = load_dataset("asset", "simplification", split="validation[:50]")
    print(f"✅ ASSET loaded: {len(asset_data)} examples")
    print(f"   Fields: {asset_data.column_names}")
except Exception as e:
    print(f"❌ ASSET error: {e}")
    # Fallback: use "full" split from ratings
    asset_data = load_dataset("asset", "ratings", split="full[:50]")
    print(f"✅ ASSET (ratings/full) loaded: {len(asset_data)} examples")

# ============================================================
# TurkCorpus - English simplification  
# ============================================================
print("\nLoading TurkCorpus...")
try:
    turk_data = load_dataset("GEM/wiki_auto_asset_turk", split="test_turk[:50]") #use test_turk for full dataset this one has many splits this is the best for eval 
    print(f"✅ TurkCorpus loaded: {len(turk_data)} examples")
    print(f"   Fields: {turk_data.column_names}")
except Exception as e:
    print(f"❌ TurkCorpus error: {e}")

# ============================================================
# DEplain - German simplification
# ============================================================
print("\nLoading DEplain...")
try:
    deplain_data = load_dataset("DEplain/DEplain-web", split="train[:50]")
    print(f"✅ DEplain loaded: {len(deplain_data)} examples")
    print(f"   Fields: {deplain_data.column_names}")
except Exception as e:
    print(f"❌ DEplain error: {e}")



Loading ASSET...
✅ ASSET loaded: 50 examples
   Fields: ['original', 'simplifications']

Loading TurkCorpus...
✅ TurkCorpus loaded: 50 examples
   Fields: ['gem_id', 'gem_parent_id', 'source', 'target', 'references']

Loading DEplain...
✅ DEplain loaded: 50 examples
   Fields: ['original', 'simplification', 'pair_id', 'complex_document_id', 'simple_document_id', 'domain', 'corpus', 'simple_url', 'complex_url', 'simple_level', 'complex_level', 'simple_location_html', 'complex_location_html', 'simple_location_txt', 'complex_location_txt', 'alignment_location', 'simple_author', 'complex_author', 'simple_title', 'complex_title', 'license', 'last_access']


Now we want to create Domain Specific test cases. This helps us differentiate from real world vs. wikipedia sentences - users will paste text from specific domains that have unique translation challenges. Often users won't paste wikipedia bt rather dense, bureaucratic texts.

These help for the following:

1. Standard benchmarks are limited like with ASSET and TurkCorpus. 
Dataset	Content	Limitation
ASSET - Wikipedia sentences	Simple factual statements, no legal/medical jargon
TurkCorpus - Wikipedia sentences	Same — general knowledge, easy vocabulary
DEplain	- German web content	Better, but may not cover all your target domains

2. Better target toward user base
Websites and documents often use complex language that excludes people with cognitive impairments, low literacy, dyslexia, seniors, and non-native speakers.
The hard cases are:
Legal — contracts, terms of service, government forms
Medical — doctor letters, medication instructions, diagnosis reports
Administrative — tax documents, immigration forms, official letters
Financial — bank statements, insurance policies

3. Better domain fit
"KlarText handles the kinds of texts that users actually paste (web, admin, health) better than a naïve 'just prompt GPT' demo."
To prove this claim, you need to test on those actual domains.

4. Different domains have different failure modes that standard benchmarks won't catch these because they don't contain such text.
* Legal - Model changes "must" to "should" (changes obligation), drops deadlines
* Medical - Model simplifies dosage ("2x daily" → "regularly"), misses warnings
* Financial - Model drops amounts, changes percentages
* Admin - Model misses deadlines, contact info, required documents

5. German specific challenges
German bureaucratic language (Behördendeutsch) has specific patterns and you often need German domain examples to test these:
* Compound nouns: Einkommensteuererklärung (income tax return)
* Passive voice: Es wird darauf hingewiesen, dass...
* Long nested sentences

How to use these tests:

* Run simplification on each domain example
* Check if critical elements are preserved (numbers, dates, obligations)
* Compare models — which one handles legal text better?
* Identify weaknesses — "Model X always drops deadlines in German legal text"

In [12]:
DOMAIN_EXAMPLES = [
    {"id": "de_legal", "text": "...", "lang": "de", "domain": "legal"},
    {"id": "de_medical", "text": "...", "lang": "de", "domain": "medical"},
    {"id": "en_legal", "text": "...", "lang": "en", "domain": "legal"},
    {"id": "en_admin", "text": "...", "lang": "en", "domain": "administrative"},
]

Step 3: Define Prompt Templates

This will use templates from prompts/templates/ but test variations:

| Strategy      | Description                                         | Expected Behavior                    |
|---------------|-----------------------------------------------------|--------------------------------------|
| Generic       | "Simplify this text"                                | Baseline, may change structure       |
| Level-Based   | Use simplify_en.txt / simplify_de.txt templates     | Controlled output by level           |
| Lexical-Only  | Replace words, keep structure                       | Minimal structural change            |
| ELI5          | Explain like I'm 5                                  | Maximum simplification               |
| Structured    | Force bullet points + key points                    |                                      |

We will test 5 different prompting strategies to see which produces the best simplifications.

In [13]:
def build_prompt(text: str, lang: str, level: str, strategy: str) -> str:
    """Build prompt based on strategy and level."""
    # Use templates from prompts/templates/
    # Return system_prompt, user_content tuple

In [14]:
# Define prompt strategies as Python strings
PROMPT_STRATEGIES = {
    "generic": "Simplify this text for a general audience:\n\n{text}",
    
    "lexical_only": """Keep the exact same sentence structure.
Only replace complex words with simpler synonyms.
Do NOT add bullet points or change the structure.

Text: {text}

Simplified:""",
    
    "eli5": "Explain this like I am 5 years old:\n\n{text}",
    
    "structured": """Convert to a scannable format with:
- 2-3 Key Points at the start
- Headings and bullet points
- Short sentences

Text: {text}""",
}

print(f"✅ Defined {len(PROMPT_STRATEGIES)} prompt strategies")

✅ Defined 4 prompt strategies


In [15]:
# Use a prompt strategy
strategy = "eli5"
test_text = "The implementation of regulatory frameworks..."

prompt = PROMPT_STRATEGIES[strategy].format(text=test_text)
# Now send `prompt` to the Groq API

Step 4: Run Baseline Experiments

| Dimension    | Values                        |
|--------------|------------------------------|
| Models       | 3 models                     |
| Languages    | EN, DE                       |
| Levels       | very_easy, easy, medium      |
| Strategies   | 5 prompt strategies          |
| Examples     | 50 per dataset               |

In [17]:
#create test set
# Combine datasets into a unified test set
# Normalize field names across datasets

test_set = []

# Add ASSET examples (English)
for item in asset_data:
    test_set.append({
        "id": f"asset_{len(test_set)}",
        "text": item["original"],
        "lang": "en",
        "domain": "wikipedia",
        "references": item["simplifications"]
    })

# Add TurkCorpus examples (English)
for item in turk_data:
    test_set.append({
        "id": f"turk_{len(test_set)}",
        "text": item["source"],
        "lang": "en",
        "domain": "wikipedia",
        "references": item["references"]
    })

# Add DEplain examples (German)
for item in deplain_data:
    test_set.append({
        "id": f"deplain_{len(test_set)}",
        "text": item["original"],
        "lang": "de",
        "domain": item.get("domain", "web"),
        "references": [item["simplification"]]
    })

# Add domain-specific examples
DOMAIN_EXAMPLES = [
    {
        "id": "de_legal",
        "text": "Der Antragsteller muss die erforderlichen Unterlagen innerhalb von 14 Tagen einreichen, andernfalls wird der Antrag abgelehnt.",
        "lang": "de",
        "domain": "legal",
        "references": []
    },
    {
        "id": "en_legal",
        "text": "The tenant shall vacate the premises no later than 30 days following written notice, failing which the landlord may pursue eviction proceedings.",
        "lang": "en",
        "domain": "legal",
        "references": []
    },
]
test_set.extend(DOMAIN_EXAMPLES)

print(f"✅ Test set created: {len(test_set)} examples")
print(f"   Languages: {set(ex['lang'] for ex in test_set)}")
print(f"   Domains: {set(ex['domain'] for ex in test_set)}")

✅ Test set created: 152 examples
   Languages: {'en', 'de'}
   Domains: {'legal', 'bible', 'wikipedia'}


In [18]:
def simplify(example: dict, model: str, level: str, strategy: str) -> tuple:
    """
    Call the LLM to simplify text.
    Returns: (output_text, latency_ms)
    """
    text = example["text"]
    lang = example["lang"]
    
    # Build the prompt based on strategy
    if strategy in PROMPT_STRATEGIES:
        prompt = PROMPT_STRATEGIES[strategy].format(text=text)
    else:
        prompt = f"Simplify this {lang} text:\n\n{text}"
    
    # Add level instruction
    level_instructions = {
        "very_easy": "Use very short sentences (max 10 words). Explain all difficult words.",
        "easy": "Use short sentences (max 15 words). Keep it clear and simple.",
        "medium": "Use plain language. Avoid jargon but keep normal sentence length."
    }
    
    system_prompt = f"You simplify text. Target level: {level}. {level_instructions.get(level, '')}"
    
    # Call the API
    start_time = time.time()
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3,
            max_tokens=1024
        )
        output = response.choices[0].message.content
        latency_ms = (time.time() - start_time) * 1000
        return output, latency_ms
    except Exception as e:
        print(f"❌ Error with {model}: {e}")
        return f"ERROR: {e}", 0

In [19]:
# Run experiments with logging
results = []
STRATEGIES = list(PROMPT_STRATEGIES.keys())  # ["generic", "lexical_only", "eli5", "structured"]

# Limit test set for initial testing (remove [:5] for full run)
test_subset = test_set[:5]

print(f"🧪 Running experiments:")
print(f"   Models: {len(MODELS_TO_TEST)}")
print(f"   Examples: {len(test_subset)}")
print(f"   Levels: {len(LEVELS)}")
print(f"   Strategies: {len(STRATEGIES)}")
print(f"   Total API calls: {len(MODELS_TO_TEST) * len(test_subset) * len(LEVELS) * len(STRATEGIES)}")
print()

for model in MODELS_TO_TEST:
    print(f"\n📊 Testing {model}...")
    for example in test_subset:
        for level in LEVELS:
            for strategy in STRATEGIES:
                output, elapsed = simplify(example, model, level, strategy)
                results.append({
                    "model": model,
                    "example_id": example["id"],
                    "input": example["text"][:100] + "...",  # Truncate for storage
                    "output": output,
                    "level": level,
                    "strategy": strategy,
                    "lang": example["lang"],
                    "domain": example["domain"],
                    "latency_ms": elapsed,
                })
                print(f"  ✓ {example['id']} | {level} | {strategy} | {elapsed:.0f}ms")

print(f"\n✅ Completed {len(results)} experiments")

🧪 Running experiments:
   Models: 3
   Examples: 5
   Levels: 3
   Strategies: 4
   Total API calls: 180


📊 Testing qwen/qwen3-32b...
  ✓ asset_0 | very_easy | generic | 1423ms
  ✓ asset_0 | very_easy | lexical_only | 1094ms
  ✓ asset_0 | very_easy | eli5 | 1094ms
  ✓ asset_0 | very_easy | structured | 1556ms
  ✓ asset_0 | easy | generic | 1000ms
  ✓ asset_0 | easy | lexical_only | 3519ms
  ✓ asset_0 | easy | eli5 | 810ms
  ✓ asset_0 | easy | structured | 1890ms
  ✓ asset_0 | medium | generic | 1533ms
  ✓ asset_0 | medium | lexical_only | 910ms
  ✓ asset_0 | medium | eli5 | 2329ms
  ✓ asset_0 | medium | structured | 1537ms
  ✓ asset_1 | very_easy | generic | 2920ms
  ✓ asset_1 | very_easy | lexical_only | 2681ms
  ✓ asset_1 | very_easy | eli5 | 1941ms
  ✓ asset_1 | very_easy | structured | 2714ms
  ✓ asset_1 | easy | generic | 2124ms
  ✓ asset_1 | easy | lexical_only | 1778ms
  ✓ asset_1 | easy | eli5 | 1885ms
  ✓ asset_1 | easy | structured | 3364ms
  ✓ asset_1 | medium | generic | 1

Alright, super cool we have a lot of... stuff and a groq dashboard that indicates this almost had us hit our rate limit. But, what does this mean, I need some simple text to figure this out.

In [20]:
#view results as a table
results_df = pd.DataFrame(results)
print(f"📊 Collected {len(results_df)} results\n")

# Show first few rows
display(results_df.head(10))

📊 Collected 180 results



,model,example_id,input,output,level,strategy,lang,domain,latency_ms
0,qwen/qwen3-32b,asset_0,"Adjacent counties are Marin (to the south), Me...","<think>\nOkay, let's see. The user wants me to...",very_easy,generic,en,wikipedia,1423.298120
1,qwen/qwen3-32b,asset_0,"Adjacent counties are Marin (to the south), Me...","<think>\nOkay, let's see. The user wants me to...",very_easy,lexical_only,en,wikipedia,1093.714952
2,qwen/qwen3-32b,asset_0,"Adjacent counties are Marin (to the south), Me...","<think>\nOkay, the user wants me to explain th...",very_easy,eli5,en,wikipedia,1093.952179
3,qwen/qwen3-32b,asset_0,"Adjacent counties are Marin (to the south), Me...","<think>\nOkay, the user wants me to convert th...",very_easy,structured,en,wikipedia,1555.933237
4,qwen/qwen3-32b,asset_0,"Adjacent counties are Marin (to the south), Me...","<think>\nOkay, let's see. The user wants me to...",easy,generic,en,wikipedia,999.747753
5,qwen/qwen3-32b,asset_0,"Adjacent counties are Marin (to the south), Me...","<think>\nOkay, let's see. The user wants me to...",easy,lexical_only,en,wikipedia,3518.867016
6,qwen/qwen3-32b,asset_0,"Adjacent counties are Marin (to the south), Me...","<think>\nOkay, the user wants me to explain th...",easy,eli5,en,wikipedia,810.487986
7,qwen/qwen3-32b,asset_0,"Adjacent counties are Marin (to the south), Me...","<think>\nOkay, the user wants me to convert th...",easy,structured,en,wikipedia,1889.706135
8,qwen/qwen3-32b,asset_0,"Adjacent counties are Marin (to the south), Me...","<think>\nOkay, let's see. The user wants me to...",medium,generic,en,wikipedia,1533.195972
9,qwen/qwen3-32b,asset_0,"Adjacent counties are Marin (to the south), Me...","<think>\nOkay, let's see. The user wants me to...",medium,lexical_only,en,wikipedia,909.666777


Hmmm... wtf is this even telling me?


In [22]:
#Average latency by model
model_summary = results_df.groupby("model").agg({
    "latency_ms": ["mean", "min", "max", "count"]
}).round(0)
model_summary.columns = ["Avg (ms)", "Min (ms)", "Max (ms)", "Count"]

print("⏱️ Latency by Model:")
display(model_summary)

⏱️ Latency by Model:


,Avg (ms),Min (ms),Max (ms),Count
model,,,,
meta-llama/llama-4-maverick-17b-128e-instruct,1586.0,160.0,4249.0,60
moonshotai/kimi-k2-instruct,1016.0,0.0,4248.0,60
qwen/qwen3-32b,2682.0,810.0,26413.0,60


Oh god, great... not better

In [23]:
# Average latency by strategy
strategy_summary = results_df.groupby("strategy").agg({
    "latency_ms": ["mean", "count"]
}).round(0)
strategy_summary.columns = ["Avg Latency (ms)", "Count"]

print("\n📝 Results by Strategy:")
display(strategy_summary)

# Average latency by level
level_summary = results_df.groupby("level").agg({
    "latency_ms": ["mean", "count"]
}).round(0)
level_summary.columns = ["Avg Latency (ms)", "Count"]


📝 Results by Strategy:


,Avg Latency (ms),Count
strategy,,
eli5,1535.0,45
generic,1498.0,45
lexical_only,1505.0,45
structured,2507.0,45


What hellhole did I end up in?

In [24]:
# Show a few example outputs to compare
def show_sample(example_id: str, level: str = "easy"):
    """Show outputs from all models for a specific example."""
    subset = results_df[
        (results_df["example_id"] == example_id) & 
        (results_df["level"] == level)
    ]
    
    print(f"{'='*80}")
    print(f"📄 Example: {example_id} | Level: {level}")
    print(f"{'='*80}")
    
    # Show original input
    original = subset.iloc[0]["input"]
    print(f"\n🔹 ORIGINAL:\n{original}\n")
    
    # Show each model × strategy output
    for _, row in subset.iterrows():
        print(f"\n🔸 {row['model'].split('/')[-1]} | {row['strategy']}")
        print(f"   Latency: {row['latency_ms']:.0f}ms")
        print(f"   Output: {row['output'][:300]}...")
        print("-" * 60)

# Show samples
show_sample("asset_0", "easy")
show_sample("asset_0", "very_easy")

📄 Example: asset_0 | Level: easy

🔹 ORIGINAL:
Adjacent counties are Marin (to the south), Mendocino (to the north), Lake (northeast), Napa (to the...


🔸 qwen3-32b | generic
   Latency: 1000ms
   Output: <think>
Okay, let's see. The user wants me to simplify this text about adjacent counties. The original sentence lists several counties with their directions relative to the main one.

First, I need to make sure the directions are clear. The original uses "to the south," "to the north," etc. Maybe I ...
------------------------------------------------------------

🔸 qwen3-32b | lexical_only
   Latency: 3519ms
   Output: <think>
Okay, let's see. The user wants me to simplify the given text. The original sentence lists adjacent counties with their directions. The task is to replace complex words with simpler synonyms while keeping the same structure.

First, "adjacent" can be "next to" or "near." But "adjacent" is al...
------------------------------------------------------------

🔸 qwen3

Finally something useful. But those responses are mostly bad.

In [25]:
# Pivot table: compare models for a specific example + level
def compare_models(example_id: str, level: str, strategy: str = "generic"):
    """Side-by-side model comparison."""
    subset = results_df[
        (results_df["example_id"] == example_id) & 
        (results_df["level"] == level) &
        (results_df["strategy"] == strategy)
    ]
    
    print(f"\n📊 Comparison: {example_id} | {level} | {strategy}\n")
    
    for _, row in subset.iterrows():
        model_name = row["model"].split("/")[-1]
        print(f"{'─'*60}")
        print(f"🤖 {model_name} ({row['latency_ms']:.0f}ms):\n")
        print(row["output"][:500])
        print()

compare_models("asset_0", "easy", "eli5")


📊 Comparison: asset_0 | easy | eli5

────────────────────────────────────────────────────────────
🤖 qwen3-32b (810ms):

<think>
Okay, the user wants me to explain the original sentence as if I'm talking to a 5-year-old. Let me start by breaking down the original text. It lists several counties adjacent to a main county, each with their direction. The user probably wants a simple explanation without complex words.

First, I need to use very short sentences, each under 15 words. Maybe start with a main county and then mention each neighbor with their direction. Use simple terms like "next to" instead of "adjacent."

────────────────────────────────────────────────────────────
🤖 llama-4-maverick-17b-128e-instruct (563ms):

Here are the neighbors of this place:

* Marin is below it.
* Mendocino is above it.
* Lake is above and to the right.
* Napa is to the right.
* Solano and Contra Costa are below and to the right.

────────────────────────────────────────────────────────────
🤖 kimi-k2-

ummm... are you all ok?

In [26]:
# Add output length as a metric
results_df["output_length"] = results_df["output"].str.len()

# Compare output length by model and level
length_summary = results_df.groupby(["model", "level"])["output_length"].mean().round(0)
print("\n📏 Average Output Length (characters):")
display(length_summary.unstack())


📏 Average Output Length (characters):


level,easy,medium,very_easy
model,,,
meta-llama/llama-4-maverick-17b-128e-instruct,164.0,249.0,269.0
moonshotai/kimi-k2-instruct,135.0,150.0,173.0
qwen/qwen3-32b,2044.0,1877.0,1970.0


I gotta come back to this, my head hurts.

In [27]:
# Save results to CSV for later analysis
results_df.to_csv("baseline_results.csv", index=False)
print("💾 Results saved to baseline_results.csv")

💾 Results saved to baseline_results.csv
